**Alunos:**\
*Bruno Berndt Lima - 12542550*\
*Fernando Gonçalves Campos - 12542352*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# !pip install torcheval
# !pip install caserecommender

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances
from sklearn.decomposition import PCA

import torch
from torch import nn, optim, cuda
from torch.utils.data import TensorDataset, DataLoader

import torcheval
import torcheval.metrics
import torcheval.metrics.regression

from caserec.recommenders.item_recommendation.userknn import UserKNN
from caserec.recommenders.rating_prediction.userknn import UserKNN as UserKNN_ratings

import matplotlib.pyplot as plt

import os

from tqdm.notebook import tqdm

from typing import Callable

In [ ]:
# WORKING_DIR: str = '.'
WORKING_DIR: str = '/content/drive/MyDrive/Trabalho Sistemas de Recomendação'

DATASET_DIR: str = f'{WORKING_DIR}/dataset'

RATINGS_FILE: str = f'{DATASET_DIR}/ratings_sample.csv'

EMBEDDINGS_DIR: str = f'{WORKING_DIR}/embeddings'

if not os.path.isdir(EMBEDDINGS_DIR):
    os.mkdir(EMBEDDINGS_DIR)

MODELS_DIR: str = f'{WORKING_DIR}/models'

if not os.path.isdir(MODELS_DIR):
    os.mkdir(MODELS_DIR)

In [ ]:
# For similarity calculation
USER_ORIGINAL_RATINGS_WEIGHT: float = 15
USER_RELATIVE_RATINGS_WEIGHT: float = 0

ITEM_ORIGINAL_RATINGS_WEIGHT: float = 15
ITEM_RELATIVE_RATINGS_WEIGHT: float = 0

In [ ]:
TEST_SIZE: float = 0.1
TRAIN_TEST_SPLIT_SEED: int = 2

In [ ]:
USER_EMBEDDINGS_SIZE: int = 1024 # In the range [1, n_users]
ITEM_EMBEDDINGS_SIZE: int = 400 # In the range [1, n_items]

USER_EMBEDDINGS_NAME: str = f'user_pca{USER_EMBEDDINGS_SIZE}'
ITEM_EMBEDDINGS_NAME: str = f'item_pca{ITEM_EMBEDDINGS_SIZE}'

USER_EMBEDDINGS_FILE: str = f'{EMBEDDINGS_DIR}/{USER_EMBEDDINGS_NAME}.npy'
ITEM_EMBEDDINGS_FILE: str = f'{EMBEDDINGS_DIR}/{ITEM_EMBEDDINGS_NAME}.npy'

FORCE_NEW_EMBEDDINGS: bool = False
SAVE_EMBEDDINGS: bool = True

In [ ]:
TRAIN_BATCH_SIZE: int = 256
TEST_BATCH_SIZE: int = 256

In [ ]:
N_EPOCHS: int = 30
LEARNING_RATE: float = 0.0005
TOWERS_FINAL_SIZE: int = 512
DROPOUT: float = 0.15

USE_TQDM: bool = True
VERBOSE_TRAINNING: bool = True

MODEL_NAME: str = f'two_tower-{USER_EMBEDDINGS_NAME}-{ITEM_EMBEDDINGS_NAME}'

MODEL_FILE: str = f'{MODELS_DIR}/{MODEL_NAME}.pt'

FORCE_TRAINNING: bool = True
SAVE_MODEL: bool = True

In [ ]:
PYTORCH_DEVICE: str = 'cuda' if cuda.is_available() else 'cpu'

print(PYTORCH_DEVICE)

### Preprocessing

In [ ]:
ratings: pd.DataFrame = pd.read_csv(RATINGS_FILE, names=['userId', 'itemId', 'rating', 'timestamp'], header=0)
ratings: pd.DataFrame = ratings[['userId', 'itemId', 'rating']]

map_users: dict[int,int] = {user: idx for idx, user in enumerate(ratings['userId'].unique())}
map_items: dict[int,int] = {item: idx for idx, item in enumerate(ratings['itemId'].unique())}

ratings['userId'] = ratings['userId'].map(map_users)
ratings['itemId'] = ratings['itemId'].map(map_items)

ratings = ratings.sort_values(['userId', 'itemId'])
ratings['rating'] /= 5

existing_users: pd.CategoricalIndex = pd.CategoricalIndex(ratings['userId'].unique())
existing_items: pd.CategoricalIndex = pd.CategoricalIndex(ratings['itemId'].unique())

ratings['userId'] = ratings['userId'].astype('category').cat.set_categories(existing_users)
ratings['itemId'] = ratings['itemId'].astype('category').cat.set_categories(existing_items)

In [ ]:
print('ratings')
display(ratings.head())

In [ ]:
print(f'number of ratings:  {len(ratings)}')
print(f'number of users:  {len(existing_users)}')
print(f'number of items:  {len(existing_items)}')

In [ ]:
train_ratings: pd.DataFrame
test_ratings: pd.DataFrame
train_ratings, test_ratings = train_test_split(ratings, test_size = TEST_SIZE, random_state = TRAIN_TEST_SPLIT_SEED)

### Creating pseudo-embeddings

#### Calculating similarities

In [ ]:
# Converts the user ratings into the list format for each user
buffer_df: pd.DataFrame = ratings[['userId', 'itemId', 'rating']].copy()
buffer_df = buffer_df.pivot_table(index='userId', columns='itemId', values='rating', observed=False, fill_value=0)
buffer_df = buffer_df.reindex(index=existing_users.to_numpy(), columns=existing_items.to_numpy(), fill_value=0)

buffer_df = buffer_df.sort_index()

train_user_ratings: np.ndarray = buffer_df.to_numpy()
print(train_user_ratings.shape)

del buffer_df

In [ ]:
def get_similarity_and_bias(ratings: np.ndarray, original_ratings_weight: float = 1, relative_ratings_weight: float = 1) -> tuple[np.ndarray, np.ndarray]:
    total_weight: float = original_ratings_weight + relative_ratings_weight
    original_ratings_weight: float = 0.5
    relative_ratings_weight: float = 0.5
    if total_weight != 0:
        original_ratings_weight = original_ratings_weight / total_weight
        relative_ratings_weight = relative_ratings_weight / total_weight

    ratings: np.ndarray = ratings.copy()

    relative_ratings: np.ma.MaskedArray = np.ma.masked_array(ratings.copy(), ratings == 0)
    bias: np.ndarray = np.ma.mean(relative_ratings, axis=1).filled(0)
    relative_ratings -= bias[:, np.newaxis]
    relative_ratings += 1

    rating_similarity: np.ndarray = cosine_similarity(ratings, ratings)
    relative_rating_similarity: np.ndarray = cosine_similarity(relative_ratings.filled(0), relative_ratings.filled(0))

    np.fill_diagonal(rating_similarity, 1)
    np.fill_diagonal(relative_rating_similarity, 1)

    similarity = (original_ratings_weight * rating_similarity) + (relative_ratings_weight * relative_rating_similarity)
    np.fill_diagonal(similarity, 1)

    return similarity, bias

In [ ]:
user_similarity: np.ndarray
user_bias: np.ndarray
user_similarity, user_bias = get_similarity_and_bias(
    ratings = train_user_ratings,
    original_ratings_weight = USER_ORIGINAL_RATINGS_WEIGHT,
    relative_ratings_weight = USER_RELATIVE_RATINGS_WEIGHT
)

In [ ]:
item_similarity: np.ndarray
item_bias: np.ndarray
item_similarity, item_bias = get_similarity_and_bias(
    ratings = train_user_ratings.T,
    original_ratings_weight = ITEM_ORIGINAL_RATINGS_WEIGHT,
    relative_ratings_weight = ITEM_RELATIVE_RATINGS_WEIGHT
)

In [ ]:
print(f'User similarity: \n{user_similarity}\n')

In [ ]:
print(f'Item similarity: \n{item_similarity}\n')

#### Selecting best users/items

In [ ]:
def pca_on_similarity(similarity: np.ndarray, k: int) -> np.ndarray:
    pca: PCA = PCA(n_components=k)
    pca.fit(similarity)
    return pca.transform(similarity)

def pca_embeddings_from_similarity(similarity: np.ndarray, bias: np.ndarray, k: int) -> np.ndarray:
    return np.hstack([pca_on_similarity(similarity, k-1), bias[:,np.newaxis]])

In [ ]:
user_embeddings: np.ndarray
item_embeddings: np.ndarray
if FORCE_NEW_EMBEDDINGS or not (os.path.isfile(USER_EMBEDDINGS_FILE) or os.path.isfile(ITEM_EMBEDDINGS_FILE)):
    user_embeddings: np.ndarray = pca_embeddings_from_similarity(user_similarity, user_bias, USER_EMBEDDINGS_SIZE)
    item_embeddings: np.ndarray = pca_embeddings_from_similarity(item_similarity, item_bias, ITEM_EMBEDDINGS_SIZE)

    if SAVE_EMBEDDINGS:
        with open(USER_EMBEDDINGS_FILE, 'wb') as f:
            np.save(f, user_embeddings)
        with open(ITEM_EMBEDDINGS_FILE, 'wb') as f:
            np.save(f, item_embeddings)

else:
    with open(USER_EMBEDDINGS_FILE, 'rb') as f:
        user_embeddings = np.load(f)
    with open(ITEM_EMBEDDINGS_FILE, 'rb') as f:
        item_embeddings = np.load(f)

### Model

#### Preparing the data

In [ ]:
def create_tensor_dataset(ratings: pd.DataFrame, user_embeddings: np.ndarray, item_embeddings: np.ndarray) -> TensorDataset:
    users: np.ndarray = ratings['userId'].to_numpy()
    items: np.ndarray = ratings['itemId'].to_numpy()

    input_user_embeddings: np.ndarray = user_embeddings[users,:]
    input_item_embeddings: np.ndarray = item_embeddings[items,:]

    output_ratings: np.ndarray = ratings['rating'].to_numpy()

    input_user_emb_tensor: torch.Tensor = torch.from_numpy(input_user_embeddings).to(dtype=torch.float32)
    input_item_emb_tensor: torch.Tensor = torch.from_numpy(input_item_embeddings).to(dtype=torch.float32)
    output_ratings_tensor: torch.Tensor = torch.from_numpy(output_ratings).to(dtype=torch.float32)

    return TensorDataset(input_user_emb_tensor, input_item_emb_tensor, output_ratings_tensor)

In [ ]:
train_loader: DataLoader = DataLoader(
    create_tensor_dataset(train_ratings, user_embeddings, item_embeddings),
    shuffle = True, batch_size = TRAIN_BATCH_SIZE
)

test_loader: DataLoader = DataLoader(
    create_tensor_dataset(test_ratings, user_embeddings, item_embeddings),
    shuffle = False, batch_size = TEST_BATCH_SIZE
)

#### Performance Metrics

In [ ]:
class Submetric:
    def __init__(self, submetric_function: Callable, *metrics_used: str):
        self.compute: Callable = submetric_function
        self.metrics_used: list[str] = list(metrics_used)

In [ ]:
class MetricsHandler:
    def __init__(self):
        self.metrics: dict[str, torcheval.metrics.Metric[torch.Tensor]] = {}
        self.submetrics: dict[str, Submetric] = {}

    def add_metric(self, metric_name: str, metric: torcheval.metrics.Metric[torch.Tensor]) -> None:
        self.metrics[metric_name] = metric

    def add_submetric(self, submetric_name: str, submetric_function: Callable, *metrics_used: str) -> None:
        self.submetrics[submetric_name] = Submetric(submetric_function, *metrics_used)

    def update(self, input: torch.Tensor, target: torch.Tensor) -> None:
        for metric in self.metrics.values():
            metric.update(input, target)

    def compute(self) -> dict[str, float]:
        computed_metrics: dict[str, float] = {metric_name: metric.compute().item() for metric_name, metric in self.metrics.items()}
        computed_submetrics: dict[str, float] = {submetric_name: submetric.compute(*[computed_metrics[metric_used] for metric_used in submetric.metrics_used]) for submetric_name, submetric in self.submetrics.items()}
        computed_metrics.update(computed_submetrics)
        return computed_metrics

    def reset(self) -> None:
        for metric in self.metrics.values():
            metric.reset()

    def used_metrics(self, include_submetrics: bool = True) -> list[str]:
        used_metrics = list(self.metrics.keys())
        if include_submetrics:
            used_metrics += list(self.submetrics.keys())

        return used_metrics

In [ ]:
def evaluate(model: torch.nn.Module, loader: DataLoader, metrics: MetricsHandler) -> dict[str, float]:
    model.eval()

    with torch.no_grad():

        user_emb: torch.Tensor
        item_emb: torch.Tensor
        expected_ratings: torch.Tensor
        for user_emb, item_emb, expected_ratings in loader:
            user_emb = user_emb.to(device=PYTORCH_DEVICE)
            item_emb = item_emb.to(device=PYTORCH_DEVICE)
            target = expected_ratings.to(device=PYTORCH_DEVICE)

            outputs: torch.Tensor = model(user_emb, item_emb)

            # Metrics are measured without the normalization
            metrics.update(outputs*5, target*5)

    computed_metrics: dict[str, float] = metrics.compute()

    metrics.reset()

    return computed_metrics

In [ ]:
metrics: MetricsHandler = MetricsHandler()
metrics.add_metric('mse', torcheval.metrics.regression.MeanSquaredError(device = PYTORCH_DEVICE))
metrics.add_metric('r2',  torcheval.metrics.regression.R2Score(device = PYTORCH_DEVICE))
metrics.add_submetric('rmse', lambda mse: np.sqrt(mse), 'mse')

#### Ranking Evaluation

In [ ]:
def ranking_precision(recommendations: pd.DataFrame, relevant_items: pd.DataFrame) -> float:
    df: pd.DataFrame = recommendations.copy()
    relevant_items = relevant_items.copy()
    relevant_items['is_relevant'] = 1

    df = df.merge(relevant_items, on=['userId', 'itemId'], how='left')
    df['is_relevant'] = df['is_relevant'].fillna(0)
    df['is_relevant'] = df['is_relevant'].astype(int)

    return df['is_relevant'].mean()

def ranking_recall(recommendations: pd.DataFrame, relevant_items: pd.DataFrame) -> float:
    df: pd.DataFrame = recommendations.copy()
    df['was_recommended'] = 1

    df = df.merge(relevant_items[relevant_items['userId'].isin(df['userId'].unique())], on=['userId', 'itemId'], how='right')
    df['was_recommended'] = df['was_recommended'].fillna(0)
    df['was_recommended'] = df['was_recommended'].astype(int)

    return df['was_recommended'].mean()

def ranking_map(recommendations: pd.DataFrame, relevant_items: pd.DataFrame) -> float:
    df: pd.DataFrame = recommendations.copy()
    relevant_items = relevant_items.copy()
    relevant_items['is_relevant'] = 1

    df = df.merge(relevant_items, on=['userId', 'itemId'], how='left')
    df['is_relevant'] = df['is_relevant'].fillna(0)
    df['is_relevant'] = df['is_relevant'].astype(int)

    n_users: int = len(df['userId'].unique())

    df['numerator'] = df.groupby('userId')['is_relevant'].cumsum()
    df['score'] = df['numerator'] / df['rank']

    df = df[df['is_relevant'] != 0]
    return df.groupby('userId')['score'].mean().sum() / n_users

ranking_metrics: dict[str, Callable] = {"Precision": ranking_precision, "Recall": ranking_recall, "MAP": ranking_map}

In [ ]:
def evaluate_ranking(recommendations: pd.DataFrame, relevant_items: pd.DataFrame, k: int | list[int], metrics: dict[str, Callable]) -> dict[str, float]:
    if isinstance(k, int):
        df: pd.DataFrame = recommendations.copy()
        df = df[df['rank'] <= k]

        relevant_items = relevant_items[['userId', 'itemId']].copy()

        return {k: metric(df, relevant_items) for k, metric in metrics.items()}

    else:
        return {f'{key}@{top_k}': value for top_k in k for key, value in evaluate_ranking(recommendations, relevant_items, top_k, metrics).items()}

#### Model

In [ ]:
class UserTower(torch.nn.Module):
    def __init__(
        self,
        user_embeddings_size: int,
        output_size: int
    ):
        super(UserTower, self).__init__()

        self.l1 = torch.nn.Linear(user_embeddings_size, 2048, dtype=torch.float32)
        self.drop1 =  torch.nn.Dropout(DROPOUT)
        self.l2 = torch.nn.Linear(2048, 2048, dtype=torch.float32)
        self.drop2 =  torch.nn.Dropout(DROPOUT)
        self.output_l = torch.nn.Linear(2048, output_size, dtype=torch.float32)

    def forward(self, user_embeddings):
        temp = self.drop1(torch.relu(self.l1(user_embeddings)))
        temp = self.drop2(torch.relu(self.l2(temp)))
        return torch.relu(self.output_l(temp))

In [ ]:
class ItemTower(torch.nn.Module):
    def __init__(
        self,
        item_embeddings_size: int,
        output_size: int
    ):
        super(ItemTower, self).__init__()

        self.l1 = torch.nn.Linear(item_embeddings_size, 2048, dtype=torch.float32)
        self.drop1 =  torch.nn.Dropout(DROPOUT)
        self.l2 = torch.nn.Linear(2048, 2048, dtype=torch.float32)
        self.drop2 =  torch.nn.Dropout(DROPOUT)
        self.output_l = torch.nn.Linear(2048, output_size, dtype=torch.float32)

    def forward(self, item_embeddings):
        temp = self.drop1(torch.relu(self.l1(item_embeddings)))
        temp = self.drop2(torch.relu(self.l2(temp)))
        return torch.relu(self.output_l(temp))

In [ ]:
class TwoTowerModel(torch.nn.Module):
    def __init__(
        self,
        user_embeddings_size: int,
        item_embeddings_size: int,
        towers_final_size: int,
    ):
        super(TwoTowerModel, self).__init__()

        self.user_tower: UserTower = UserTower(user_embeddings_size, towers_final_size)
        self.item_tower: ItemTower = ItemTower(item_embeddings_size, towers_final_size)

    def forward(
        self,
        user_embeddings,
        item_embeddings
    ):
        user_temp: torch.Tensor = self.user_tower(user_embeddings)
        item_temp: torch.Tensor = self.item_tower(item_embeddings)

        return torch.sum(user_temp * item_temp, dim=1)

#### Training

In [ ]:
model: torch.nn.Module
loss_fn: torch.nn.MSELoss = torch.nn.MSELoss(reduction='mean')

if FORCE_TRAINNING or not os.path.isfile(MODEL_FILE):
    model = TwoTowerModel(
        user_embeddings_size = USER_EMBEDDINGS_SIZE,
        item_embeddings_size = ITEM_EMBEDDINGS_SIZE,
        towers_final_size =    TOWERS_FINAL_SIZE
    )
    model.to(device=PYTORCH_DEVICE)

    optimizer: torch.optim.Adam = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)

    try:
        for epoch in (tqdm(range(1, N_EPOCHS+1), desc='Training', unit='epoch') if USE_TQDM else range(1, N_EPOCHS+1)):
            if VERBOSE_TRAINNING or not USE_TQDM:
                print(f'current epoch: {epoch} / {N_EPOCHS}', end=('\n' if VERBOSE_TRAINNING else '\r'))

            model.train()

            epoch_loss = 0

            user_emb: torch.Tensor
            item_emb: torch.Tensor
            expected_ratings: torch.Tensor
            for user_emb, item_emb, expected_ratings in train_loader:
                optimizer.zero_grad()

                user_emb = user_emb.to(device=PYTORCH_DEVICE)
                item_emb = item_emb.to(device=PYTORCH_DEVICE)
                expected_ratings = expected_ratings.to(device=PYTORCH_DEVICE)

                outputs: torch.Tensor = model(user_emb, item_emb)
                loss: torch.Tensor = torch.sqrt(loss_fn(outputs, expected_ratings))

                loss.backward()
                optimizer.step()

                epoch_loss += np.square(loss.item()) * len(user_emb)

            if VERBOSE_TRAINNING:
                # RMSE assuming normalized input
                tqdm.write(f'epoch loss: {np.sqrt((epoch_loss * 25) / len(train_loader.dataset))}')
                tqdm.write(f'test loss: {evaluate(model, test_loader, metrics)["rmse"]}\n')

    except KeyboardInterrupt:
        pass

    if not (VERBOSE_TRAINNING or USE_TQDM):
        print('')

    if SAVE_MODEL:
        torch.save(model, MODEL_FILE)

else:
    model = torch.load(MODEL_FILE)
    model.to(device = PYTORCH_DEVICE)

### Validation

In [ ]:
PRECISION: int = 5
MAX_METRIC_NAME_LENGTH: int = len(max(metrics.used_metrics(), key=len))

computed_train_eval: dict[str, float] = evaluate(model, train_loader, metrics)
print('Train Eval:')
print('\n'.join(f'{metric_name:<{MAX_METRIC_NAME_LENGTH}} : {metric_eval:.{PRECISION}f}' for metric_name, metric_eval in computed_train_eval.items()))

computed_test_eval: dict[str, float] = evaluate(model, test_loader, metrics)
print('\nTest Eval:')
print('\n'.join(f'{metric_name:<{MAX_METRIC_NAME_LENGTH}} : {metric_eval:.{PRECISION}f}' for metric_name, metric_eval in computed_test_eval.items()))

In [ ]:
def get_predictions(model: torch.nn.Module, df: pd.DataFrame, user_embeddings: np.ndarray, item_embeddings: np.ndarray) -> pd.DataFrame:
    predictions: pd.DataFrame = pd.DataFrame({"userId": [], "itemId": [], "prediction": []})
    predictions['userId'] = predictions['userId'].astype(int)
    predictions['itemId'] = predictions['itemId'].astype(int)

    items_set: set[int] = set(df['itemId'])
    model.eval()

    with torch.no_grad():
        for user in tqdm(df['userId'].unique()):
            user_df: pd.DataFrame = df[df['userId'] == user]
            new_items_ids: list[int] = list(items_set - set(user_df['itemId']))

            input_user_embeddings: torch.Tensor = torch.from_numpy(np.repeat(user_embeddings[user][np.newaxis, :], len(new_items_ids), axis=0)).to(device=PYTORCH_DEVICE, dtype=torch.float32)
            input_item_embeddings: torch.Tensor = torch.from_numpy(item_embeddings[new_items_ids,:]).to(device=PYTORCH_DEVICE, dtype=torch.float32)

            user_predictions: list[float] = model(input_user_embeddings, input_item_embeddings).squeeze().tolist()

            predictions = pd.concat([predictions, pd.DataFrame({"userId": user, "itemId": new_items_ids, "prediction": user_predictions})])

    predictions['rank'] = predictions.groupby("userId")['prediction'].rank('first', ascending=False)
    predictions['rank'] = predictions['rank'].astype(int)
    predictions = predictions.sort_values(["userId", "rank"])
    return predictions

ranked_predictions: pd.DataFrame = get_predictions(model, train_ratings, user_embeddings, item_embeddings)
display(ranked_predictions.head())

In [ ]:
evaluate_ranking(ranked_predictions, test_ratings, k = [5, 10, 20], metrics = ranking_metrics)

### User KNN for comparison

In [ ]:
TRAIN_RATINGS_FILE: str = f'train.csv'
TEST_RATINGS_FILE: str = f'test.csv'
USERKNN_OUT_FILE: str = f'{WORKING_DIR}/userknn_out.csv'

KNN for rating prediction

In [ ]:
# Going back to the original ratings
train_ratings['rating'] *= 5
test_ratings['rating'] *= 5
train_ratings.to_csv(TRAIN_RATINGS_FILE, index=False, header=False, sep='\t')
test_ratings.to_csv(TEST_RATINGS_FILE, index=False, header=False, sep='\t')
train_ratings['rating'] /= 5
test_ratings['rating'] /= 5

UserKNN_ratings(TRAIN_RATINGS_FILE, TEST_RATINGS_FILE).compute()

KNN for item recommendation

In [ ]:
train_ratings.to_csv(TRAIN_RATINGS_FILE, index=False, header=False, sep='\t')
test_ratings.to_csv(TEST_RATINGS_FILE, index=False, header=False, sep='\t')
if not os.path.isfile(USERKNN_OUT_FILE):
    UserKNN(TRAIN_RATINGS_FILE, TEST_RATINGS_FILE, output_file=USERKNN_OUT_FILE, as_similar_first=False, rank_length=20).compute() # Creates the predictions for the any pair that didn't appear on the train data

In [ ]:
userknn_predictions: pd.DataFrame = pd.read_csv(USERKNN_OUT_FILE, names=['userId', 'itemId', 'prediction'], sep='\t', header=0)

userknn_predictions['rank'] = userknn_predictions.groupby("userId")['prediction'].rank('first', ascending=False)
userknn_predictions['rank'] = userknn_predictions['rank'].astype(int)
userknn_predictions = userknn_predictions.sort_values(["userId", "rank"])

display(userknn_predictions.head())

In [ ]:
display(userknn_predictions)

In [ ]:
evaluate_ranking(userknn_predictions, test_ratings, k = [5, 10, 20], metrics = ranking_metrics)

### Comparison

In [ ]:
# Evaluation considering only items that each user didn't dislike as being relevant
ranking_eval = evaluate_ranking(ranked_predictions, test_ratings[test_ratings['rating'] >= 0.6], k = [5, 10, 20], metrics = ranking_metrics)
knn_eval = evaluate_ranking(userknn_predictions, test_ratings[test_ratings['rating'] >= 0.6], k = [5, 10, 20], metrics = ranking_metrics)

In [ ]:
display(pd.DataFrame({'model': ranking_eval, 'knn': knn_eval}))